In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')

In [5]:
def generate_condition(means = [1.1,1.5,2.1,2.9],
                       stds = [.8,.7,.9,.8],
                       n = 10 ,
                       condition = 1,
                       plot = True):
    temp = [];condition = []; time = [];# phase in one condition
    for ii,(m,s) in enumerate(zip(means,stds)):
        temp.append(np.random.normal(m,s,size=int(n)))
        condition.append([condition]*n)
        time.append([ii+1]*n)
    df = pd.DataFrame(np.vstack([np.concatenate(temp),
               np.concatenate(condition),
               np.concatenate(time)]).T,
                     columns = ['measure','condition','time'])
    df['sub'] = np.tile(np.arange(1,n+1),len(means))
    if plot:
        sns.catplot(x='time',y='measure',data=df1,kind = 'point')

In [ ]:
n = 10
means = [[1.1,1.5,2.1,2.9],
         [1.2,1.4,2.4,3.1],
         [1.0,1.2,2.1,2.8],
         [1.1,1.2,2.3,2.8]]
stds = [[.82,.7,.96,.82],
        [.85,.77,.91,.85],
        [.81,.74,.95,.83],
        [.8,.7,.9,.8]]
df = []
for ii,(mean,std) in enumerate(zip(means,stds)):
    df_temp = generate_condition(mean,std,
                                 n = n,
                                 condition = ii + 1)
    df.append(df_temp)
df = pd.concat(df)